In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
import re
from nltk.tokenize import WhitespaceTokenizer
from tqdm.auto import tqdm
import os
import sys
import torch
import csv
import numpy as np
import random
import time
import datetime
import pprint
import tensorflow as tf
from pathlib import Path
from preprocess_c import *

In [2]:

from pathlib import Path
train_path  = 'data/data_propoganda/data/protechn_corpus_eval/train'
test_path = 'data/data_propoganda/data/protechn_corpus_eval/test'
dev_path = 'data/data_propoganda/data/protechn_corpus_eval/dev'


In [3]:
def make_dset(path):
    path_ = Path(path)
    a = make_dataset(path_)
    df_1 = pd.DataFrame(columns=['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'])
    for dm in a:
        df_t = pd.DataFrame(dm,columns =['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'] )
        df_1 = df_1.append(df_t,ignore_index= True)
    return df_1.iloc[:,:-2]

In [ ]:
df_train = make_dset(train_path)
#df_train= pd.read_csv('Context_only (1).csv')
df_test = make_dset(test_path)

In [ ]:
bin = [0 if i == 'O' else 1 for i in df_train.prop.values ]
df_train['binary'] = bin


In [ ]:
mapping = {'Loaded_Language':1,'Name_Calling,Labeling':2,'Repetition':3,
           'Exaggeration,Minimisation':4,'Doubt':5,'Appeal_to_fear-prejudice':6,'Flag-Waving':7,'Causal_Oversimplification':8,
           'Slogans':9,'Appeal_to_Authority':10,'Black-and-White_Fallacy':11,'Thought-terminating_Cliches':12,'Whataboutism':13,
           'Reductio_ad_hitlerum':14,'Red_Herring':15,'Bandwagon':16,'Obfuscation,Intentional_Vagueness,Confusion':17,'Straw_Men':18,'O':0}
#df_train = df_train[df_train.binary !=0]

df_train['prop_1'] = df_train.prop.apply(lambda x: mapping[x])
df_test['prop_1'] = df_test.prop.apply(lambda x: mapping[x])
df_train

In [ ]:
import glob
import os
train_direct = glob.glob('data/data_propoganda/data/protechn_corpus_eval/train/*.txt')
articles = []
def read_articles():
  for filename in train_direct:
    myfile = open(filename)
    article = myfile.read()
    articles.append(article)
    myfile.close()
  article_ids = []
  for filename in train_direct:
    article_ids.append(filename[60:-4])
  
  return articles, article_ids
articles,art_ids = read_articles()
id2art ={i:a for a,i in zip(articles,art_ids)}

In [ ]:
def get_context(article, span, mode='sentence'):
  article = id2art[article]
  def get_num_words(sentence):
    return len(sentence.split(' '))
  if mode == "title":
    return article.split('\n')[0]
  if mode == "sentence":
    WORD_LEN_LIMIT = 120
    li = span[0]
    ri = span[1]
    span_text = article[li: ri]
    num_words = get_num_words(span_text)
    if num_words >= WORD_LEN_LIMIT:
      return span_text
    remaining_len = WORD_LEN_LIMIT - num_words
    lhs_words = remaining_len // 2
    rhs_words = remaining_len - lhs_words
    li -= 1
    lcount = 0
    while li >= 0 and article[li-1] != '\n' and lcount < lhs_words:
      if article[li] == ' ':
        lcount += 1
      li -= 1
    ri += 1
    rcount = 0
    while ri < len(article) and article[ri] != '\n' and rcount < rhs_words:
      if article[ri] == ' ':
        rcount += 1
      ri += 1
    return article[li+1: ri - 1] 

  return ""
spans_1 = [(i,k,j) for i,k,j in zip(df_train.id,df_train.start_sent,df_train.end_sent)]
df_train['context'] = [get_context(i,(s,e)) for i,s,e in spans_1]

In [ ]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import BertTokenizer

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-05


In [13]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.full_sent[index])
        title = " ".join(title.split())
        context = str(self.data.context[index])
        context = " ".join(context.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        tokenized_context = self.tokenizer.encode_plus(context,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                            return_attention_mask=True,truncation = True)

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        c_ids = tokenized_context['input_ids']
        c_mask = tokenized_context['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.prop_1[index], dtype=torch.long),
            'c_ids':torch.tensor(c_ids, dtype=torch.long),
            'c_mask': torch.tensor(c_mask,dtype= torch.long)
        } 
    
    def __len__(self):
        return self.len

In [14]:
train_size = 0.8
train_dataset=df_train.sample(frac=train_size,random_state=200)
test_dataset=df_train.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (15750, 10)
TRAIN Dataset: (12600, 10)
TEST Dataset: (3150, 10)


In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [16]:
from torch.nn import CrossEntropyLoss, MSELoss,BCEWithLogitsLoss

class ContextualBertForSequenceClassification(torch.nn.Module):
  
  def __init__(self, num_labels, ContextModel, SpanModel):
    super(ContextualBertForSequenceClassification, self).__init__()
    self.ContextModel = ContextModel
    self.SpanModel = SpanModel
    self.num_labels = num_labels

    # self.classifier = torch.nn.Linear(768*2, num_labels)
    # self.classifier1 = torch.nn.Linear(768, num_labels)
    self.classifier2 = torch.nn.Linear(768+128, num_labels)
    self.reduce_classifier = torch.nn.Linear(768, 128)
    self.dropout = torch.nn.Dropout(0.1)

  def forward(
      self,
      span_input_ids,
      span_attention_mask,
      context_input_ids,
      context_attention_mask,
      labels=None
  ):
    context_outputs = self.ContextModel(
        input_ids=context_input_ids,
        attention_mask=context_attention_mask
    )
    context_outputs = context_outputs[1] # pooler output
    span_outputs = self.SpanModel(
        input_ids=span_input_ids,
        attention_mask=span_attention_mask
    )
    span_outputs = span_outputs[1]

    context_outputs = self.reduce_classifier(context_outputs)
    pooled_output = torch.cat((span_outputs, context_outputs), axis=1)

    pooled_output = self.dropout(pooled_output)

    logits = self.classifier2(pooled_output)
    outputs = (logits,)
    if labels is not None:
      if self.num_labels == 1:
        loss_fct = MSELoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))
      else:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
      outputs = (loss,) + outputs

    return outputs

In [17]:

def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [18]:
def train(model, epochs=5):
  loss_values = []
  for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(training_loader):
      if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(training_loader), elapsed))
      b_input_ids = batch['ids'].to(device)
      b_labels = batch['targets'].to(device, dtype = torch.long)
      b_input_mask = batch['mask'].to(device)
      b_c_input_ids = batch['c_ids'].to(device)
      b_c_input_mask = batch['c_mask'].to(device)
      model.zero_grad()        
      outputs = model(b_input_ids, 
                      b_input_mask,
                      b_c_input_ids, 
                      b_c_input_mask, 
                      labels=b_labels)
      loss = outputs[0]
      total_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step() # TODO
      stats = '[%d/%d][%d/%d]\tLoss: %.4f' \
                % (epoch_i+1, epochs, step, len(training_loader), loss.item())
      print('\r' + stats, end="")
      sys.stdout.flush()
    avg_train_loss = total_loss / len(training_loader)            
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in testing_loader:
      # batch = tuple(t.to(device) for t in batch)
      b_input_ids = batch['ids'].to(device)
      b_labels = batch['targets'].to(device, dtype = torch.long)
      b_input_mask = batch['mask'].to(device)
      b_c_input_ids = batch['c_ids'].to(device)
      b_c_input_mask = batch['c_mask'].to(device)
      with torch.no_grad():        
        outputs = model(b_input_ids, 
                        b_input_mask,
                        b_c_input_ids, 
                        b_c_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy
      nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
  print("")
  print("Training complete!")
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
def get_model_predictions(model, dataloader):
  model.eval()
  predictions , true_labels = [], []
  nb_eval_steps = 0
  for batch in dataloader:
    b_input_ids = batch['ids'].to(device)
    b_labels = batch['targets'].to(device)
    b_input_mask = batch['mask'].to(device)
    b_c_input_ids = batch['c_ids'].to(device)
    b_c_input_mask = batch['c_mask'].to(device)
    with torch.no_grad():        
      logits = model(b_input_ids, 
                     b_input_mask,
                     b_c_input_ids, 
                     b_c_input_mask)
    
    logits = logits[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    pred_label = np.argmax(logits, axis=1) #[1 if a >0.5 else 0 for a in logits]
    predictions.extend(pred_label)
    true_labels.extend(label_ids)
  return predictions, true_labels

In [20]:
def get_dev_predictions(model):
  test_articles, _ = read_articles("dev-articles")
  test_spans, test_techniques = read_test_spans()

  test_articles = test_articles[1:]
  test_dataloader = get_data(test_articles, test_spans, test_techniques)
  pred, _ = get_model_predictions(model, test_dataloader)

  with open('predictions.txt', 'w') as fp:
    label_file = os.path.join(data_dir, "dev-task-TC-template.out")
    myfile = open(label_file)
    prev_index = -1
    tsvreader = csv.reader(myfile, delimiter="\t")
    for i, row in enumerate(tsvreader):
      fp.write(row[0] + '\t' + distinct_techniques[pred[i]] + '\t' + row[2] + '\t' + row[3] + '\n')

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
#from transformers import RobertaModel
from transformers import BertModel
#from transformers import RobertaForSequenceClassification
import time,sys

model_name = 'bert-base-cased'
context_model = BertModel.from_pretrained(model_name)
span_model = BertModel.from_pretrained(model_name)
model = ContextualBertForSequenceClassification(19, context_model, span_model)
model.cuda()
"""model.load_state_dict(torch.load('best.pth'))
model.eval()"""
optimizer = AdamW(model.parameters(),lr = 3e-5,eps = 1e-8) # ler = 5e-5
epochs = 7

total_steps = len(training_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
train(model, epochs=epochs)

In [ ]:
#load in model if saved. ('best.pth' is the model name)
"""model.load_state_dict(torch.load('best.pth'))
model.eval()"""

#predict and get classification report
"""pred,true = get_model_predictions(model,testing_loader)
from sklearn.metrics import classification_report
print(classification_report(pred,true))"""